<a href="https://colab.research.google.com/github/DanielaNik/lunar-landings-app/blob/master/NASA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#start here
#shift+enter in this box will run the code that's here and open a new code-box below
#import ....

#Note: Imports for the modules that we need
import numpy as np
import pandas as pd

#Note: Requests is a simple, yet elegant, HTTP library.
#Resource: https://pypi.org/project/requests/
import requests

#Note: Beautiful Soup is a library that makes it easy to scrape information from web pages.
#Note: In the following resource you can find a way to select any part of a HTML content.
#Resource: https://pypi.org/project/beautifulsoup4/
from bs4 import BeautifulSoup

#Note: IPython is a command shell for interactive computing in Python.
#Note: IPython.display is a module for display tools in IPython.
#Resource: https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html
from IPython.display import HTML


#Note: We can disable warnings for our requests.
import warnings

requests.packages.urllib3.disable_warnings()
warnings.filterwarnings("ignore")

import re
import json

In [ ]:
#Web Scarping from NASA

scraping_url = 'https://www.lroc.asu.edu/featured_sites/'

In [ ]:
#Response from site
response = requests.get(scraping_url)
response

<Response [200]>

In [ ]:
#Parsing with BeautifulSoup

raw_html = response.text
html = BeautifulSoup(raw_html, "html.parser")
type(html)

bs4.BeautifulSoup

In [ ]:
#Save in Drive

from google.colab import drive

drive.mount('/content/drive')
path = "/content/drive/MyDrive/OpenData-Project"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
html.select_one('.modal-header > h4').text

'GRAIL A'

In [ ]:
def get_data(url, path):
  response = requests.get(url)
  html = BeautifulSoup(response.text, "html.parser")

  shuttle_items = html.find_all('div', class_='modal-content')
  lon_items = html.select('.lat-lon')

  print(len(shuttle_items))
  shuttles = {}

  for i in range(0, len(shuttle_items)):
    # item = {}

    shuttle_name = shuttle_items[i].select_one('.modal-header > h4').text
    # item['Name'] = shuttle_name

    shuttle_longitude = lon_items[i].text
    # item['Longitude'] = shuttle_longitude

    shuttle_country = shuttle_items[i].select_one('.modal-body > p:nth-child(1)').text
    # item['Country'] = shuttle_country

    shuttle_description= shuttle_items[i].select_one('.modal-body > p:nth-child(2)').text
    # item['Description'] = shuttle_description

    item = {
        'Name': shuttle_name,
        'Longitude': shuttle_longitude,
        'Country': shuttle_country,
        'Description': shuttle_description
    }

    shuttles= json.dumps(item)


  df = pd.DataFrame(shuttles)
  # url_parts = url.split("/")
  # file_name = path + "/results_" + url_parts[3] + ".csv"
  df.to_json(r'/content/drive/MyDrive/OpenData-Project/Shuttles.json')

  return df

In [ ]:
nasa_shuttles_url = "https://www.lroc.asu.edu/featured_sites/"
nasa_data = get_data(nasa_shuttles_url,path)

36


ValueError: ignored

In [ ]:

print(nasa_data)

                      Name                                          Longitude  \
0                  GRAIL A  \n                  Longitude: 333.41° \n     ...   
1                  GRAIL B  \n                  Longitude: 333.17° \n     ...   
2                    LADEE  \n                  Longitude: 266.75° \n     ...   
3   Chang'e 3 Landing Site  \n                  Longitude: 340.49° \n     ...   
4   Chang'e 4 Landing Site  \n                  Longitude: 177.59° \n     ...   
5      17 March 2013 Event  \n                  Longitude: 335.67° \n     ...   
6           New Crater 260  \n                  Longitude: 319.50° \n     ...   
7           New Crater 278  \n                  Longitude: 125.41° \n     ...   
8           New Crater 572  \n                  Longitude: 30.54° \n      ...   
9                Apollo 11  \n                  Longitude: 23.47° \n      ...   
10               Apollo 12  \n                  Longitude: 336.58° \n     ...   
11               Apollo 14  